In [1]:
#Natural Language Toolkit for Natural Language Processing
!pip install autocorrect

Import neccasary libraries and import data from github

In [2]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import string
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
comment_words = ' '
stopwords = set(STOPWORDS) 
from autocorrect import Speller
from nltk.tokenize import WhitespaceTokenizer
from nltk import word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
df_hiv_post = pd.read_csv("https://raw.githubusercontent.com/SrushtiChauhan/Test-2022/main/HIV_comments_main.csv", 
                          on_bad_lines='skip');
df_hiv_comment = pd.read_csv("https://raw.githubusercontent.com/SrushtiChauhan/Test-2022/main/HIV_comments_individual.csv", 
                             on_bad_lines='skip');

In [4]:
print('HIV Posts data:', df_hiv_post.shape)
print('HIV Comments data:', df_hiv_comment.shape)

HIV Posts data: (845, 5)
HIV Comments data: (3861, 4)


In [5]:
#Data Cleaning
df_hiv_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              845 non-null    object
 1   Post_Text       845 non-null    object
 2   Total_Comments  845 non-null    int64 
 3   Score           845 non-null    int64 
 4   Post_Created    845 non-null    object
dtypes: int64(2), object(3)
memory usage: 33.1+ KB


In [6]:
df_hiv_comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               3861 non-null   object
 1   Comment          3861 non-null   object
 2   Comment_Created  3861 non-null   object
 3   Score            3861 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 120.8+ KB


In [7]:
df_hiv_post.describe()

,Total_Comments,Score
count,845.000000,845.000000
mean,4.565680,1.669822
std,7.190338,2.416590
min,0.000000,0.000000
25%,0.000000,1.000000
50%,2.000000,1.000000
75%,6.000000,2.000000
max,81.000000,28.000000


In [8]:
df_hiv_comment.describe()

,Score
count,3861.000000
mean,1.164724
std,0.686893
min,-10.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,8.000000


In [9]:
df_hiv_post.isnull().sum()

ID                0
Post_Text         0
Total_Comments    0
Score             0
Post_Created      0
dtype: int64

In [10]:
df_hiv_comment.isnull().sum()

ID                 0
Comment            0
Comment_Created    0
Score              0
dtype: int64

In [11]:
# drop all rows with any NaN and NaT values
df_hiv_post = df_hiv_post.dropna()

In [12]:
# drop all rows with any NaN and NaT values
df_hiv_comment = df_hiv_comment.dropna()

In [13]:
df_hiv_post.isnull().sum()

ID                0
Post_Text         0
Total_Comments    0
Score             0
Post_Created      0
dtype: int64

In [14]:
df_hiv_comment.isnull().sum()

ID                 0
Comment            0
Comment_Created    0
Score              0
dtype: int64

In [15]:
# Selecting duplicate rows except first
# occurrence based on all columns
duplicate = df_hiv_post[df_hiv_post.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,ID,Post_Text,Total_Comments,Score,Post_Created


In [16]:
# Selecting duplicate rows except first
# occurrence based on all columns
duplicate = df_hiv_comment[df_hiv_comment.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,ID,Comment,Comment_Created,Score


In [17]:
#Data Pre-Processing
df_hiv_post.columns

Index(['ID', 'Post_Text', 'Total_Comments', 'Score', 'Post_Created'], dtype='object')

In [18]:
#Data Pre-Processing
df_hiv_comment.columns

Index(['ID', 'Comment', 'Comment_Created', 'Score'], dtype='object')

In [19]:
df_hiv_post.head(3)

,ID,Post_Text,Total_Comments,Score,Post_Created
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00


In [20]:
df_hiv_comment.head(3)

,ID,Comment,Comment_Created,Score
0,twwkp4,Het a test at 3 months. Or a viral load at thi...,2022-04-05 15:33:54,2
1,twwkp4,Het a test at 3 months. Or a viral load at thi...,2022-04-05 15:34:11,1
2,twvhhs,Yes it is,2022-04-05 14:06:11,1


In [21]:
#Convert comments data into lower case
df_hiv_post['clean_text'] = df_hiv_post["Post_Text"].str.lower()
df_hiv_post

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv 1/2 antibody test (finger pr...
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...
...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,"carexo.com: get tested. share your hiv status,..."
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex (hiv home test) - not for those squea...
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me freaking out two days a...


In [22]:
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

In [23]:
#Remove punctuation and special characters from text data
df_hiv_post["clean_text"] = [clean_text(t) for t in df_hiv_post['clean_text']]
df_hiv_post

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger prick b...
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...
...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me freaking out two days a...


In [24]:
#Autocorrect and spelling mistake correction
spell = Speller(lang='en')

df_hiv_post["clean_text"] = [spell(t) for t in df_hiv_post["clean_text"]]
df_hiv_post

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...
...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...


In [25]:
#Tokenization
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

df_hiv_post["tokenization"] = [tk.tokenize(t) for t in df_hiv_post["clean_text"]]
df_hiv_post

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps,"[i, hope, this, helps]"
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...,"[hiv, risk, i, was, just, wondering, what, is,..."
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...,"[i, did, a, rapid, hiv, antibody, test, finger..."
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...,"[possible, hiv, symptoms, i, had, unprotected,..."
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...,"[microtears, on, penis, and, oral, sex, i, jus..."
...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...,"[carexocom, get, tested, share, your, hiv, sta..."
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...,"[child, taken, away, from, mother, for, missin..."
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...,"[safe, sex, hiv, home, test, not, for, those, ..."
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...,"[exposure, to, hiv, has, me, breaking, out, tw..."


In [26]:
from nltk.corpus import stopwords

In [27]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
df_hiv_post['remove_stopwords'] = df_hiv_post['tokenization'].apply(lambda x: [w for w in x if not w in stopwords.words('english')]) # Stem every word.
df_hiv_post # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,remove_stopwords
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps,"[i, hope, this, helps]","[hope, helps]"
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...,"[hiv, risk, i, was, just, wondering, what, is,...","[hiv, risk, wondering, risk, bleeding, hemorrh..."
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...,"[i, did, a, rapid, hiv, antibody, test, finger...","[rapid, hiv, antibody, test, finger, price, bl..."
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...,"[possible, hiv, symptoms, i, had, unprotected,...","[possible, hiv, symptoms, unprotected, sex, ma..."
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...,"[microtears, on, penis, and, oral, sex, i, jus...","[microtears, penis, oral, sex, got, blowjob, l..."
...,...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...,"[carexocom, get, tested, share, your, hiv, sta...","[carexocom, get, tested, share, hiv, status, p..."
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...,"[child, taken, away, from, mother, for, missin...","[child, taken, away, mother, missing, doctors,..."
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...,"[safe, sex, hiv, home, test, not, for, those, ...","[safe, sex, hiv, home, test, squeamish, blood]"
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...,"[exposure, to, hiv, has, me, breaking, out, tw...","[exposure, hiv, breaking, two, days, ago, outs..."


In [29]:
ps = PorterStemmer()

In [30]:
#Stemming
df_hiv_post['stemming'] = df_hiv_post['remove_stopwords'].apply(lambda x: [ps.stem(y) for y in x]) # Stem every word.
df_hiv_post # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,remove_stopwords,stemming
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps,"[i, hope, this, helps]","[hope, helps]","[hope, help]"
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...,"[hiv, risk, i, was, just, wondering, what, is,...","[hiv, risk, wondering, risk, bleeding, hemorrh...","[hiv, risk, wonder, risk, bleed, hemorrhoid, u..."
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...,"[i, did, a, rapid, hiv, antibody, test, finger...","[rapid, hiv, antibody, test, finger, price, bl...","[rapid, hiv, antibodi, test, finger, price, bl..."
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...,"[possible, hiv, symptoms, i, had, unprotected,...","[possible, hiv, symptoms, unprotected, sex, ma...","[possibl, hiv, symptom, unprotect, sex, man, f..."
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...,"[microtears, on, penis, and, oral, sex, i, jus...","[microtears, penis, oral, sex, got, blowjob, l...","[microtear, peni, oral, sex, got, blowjob, lik..."
...,...,...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...,"[carexocom, get, tested, share, your, hiv, sta...","[carexocom, get, tested, share, hiv, status, p...","[carexocom, get, test, share, hiv, statu, prot..."
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...,"[child, taken, away, from, mother, for, missin...","[child, taken, away, mother, missing, doctors,...","[child, taken, away, mother, miss, doctor, app..."
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...,"[safe, sex, hiv, home, test, not, for, those, ...","[safe, sex, hiv, home, test, squeamish, blood]","[safe, sex, hiv, home, test, squeamish, blood]"
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...,"[exposure, to, hiv, has, me, breaking, out, tw...","[exposure, hiv, breaking, two, days, ago, outs...","[exposur, hiv, break, two, day, ago, outsid, s..."


In [31]:
lemmatizer = WordNetLemmatizer()

In [32]:
#Lemmatization
# a denotes adjective in "pos"
df_hiv_post['lemmatizing'] = df_hiv_post['stemming'].apply(lambda x: [lemmatizer.lemmatize(y, pos ="a") for y in x]) # Stem every word.
df_hiv_post # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,remove_stopwords,stemming,lemmatizing
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps,"[i, hope, this, helps]","[hope, helps]","[hope, help]","[hope, help]"
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...,"[hiv, risk, i, was, just, wondering, what, is,...","[hiv, risk, wondering, risk, bleeding, hemorrh...","[hiv, risk, wonder, risk, bleed, hemorrhoid, u...","[hiv, risk, wonder, risk, bleed, hemorrhoid, u..."
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...,"[i, did, a, rapid, hiv, antibody, test, finger...","[rapid, hiv, antibody, test, finger, price, bl...","[rapid, hiv, antibodi, test, finger, price, bl...","[rapid, hiv, antibodi, test, finger, price, bl..."
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...,"[possible, hiv, symptoms, i, had, unprotected,...","[possible, hiv, symptoms, unprotected, sex, ma...","[possibl, hiv, symptom, unprotect, sex, man, f...","[possibl, hiv, symptom, unprotect, sex, man, f..."
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...,"[microtears, on, penis, and, oral, sex, i, jus...","[microtears, penis, oral, sex, got, blowjob, l...","[microtear, peni, oral, sex, got, blowjob, lik...","[microtear, peni, oral, sex, got, blowjob, lik..."
...,...,...,...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...,"[carexocom, get, tested, share, your, hiv, sta...","[carexocom, get, tested, share, hiv, status, p...","[carexocom, get, test, share, hiv, statu, prot...","[carexocom, get, test, share, hiv, statu, prot..."
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...,"[child, taken, away, from, mother, for, missin...","[child, taken, away, mother, missing, doctors,...","[child, taken, away, mother, miss, doctor, app...","[child, taken, away, mother, miss, doctor, app..."
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...,"[safe, sex, hiv, home, test, not, for, those, ...","[safe, sex, hiv, home, test, squeamish, blood]","[safe, sex, hiv, home, test, squeamish, blood]","[safe, sex, hiv, home, test, squeamish, blood]"
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...,"[exposure, to, hiv, has, me, breaking, out, tw...","[exposure, hiv, breaking, two, days, ago, outs...","[exposur, hiv, break, two, day, ago, outsid, s...","[exposur, hiv, break, two, day, ago, outsid, s..."


In [33]:
#combine tokenize words into sentance
df_hiv_post["post-text"] = [' '.join(t) for t in df_hiv_post["lemmatizing"]]
df_hiv_post

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,remove_stopwords,stemming,lemmatizing,post-text
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,i hope this helps,"[i, hope, this, helps]","[hope, helps]","[hope, help]","[hope, help]",hope help
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk i was just wondering what is my risk ...,"[hiv, risk, i, was, just, wondering, what, is,...","[hiv, risk, wondering, risk, bleeding, hemorrh...","[hiv, risk, wonder, risk, bleed, hemorrhoid, u...","[hiv, risk, wonder, risk, bleed, hemorrhoid, u...",hiv risk wonder risk bleed hemorrhoid use toil...
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,i did a rapid hiv antibody test finger price b...,"[i, did, a, rapid, hiv, antibody, test, finger...","[rapid, hiv, antibody, test, finger, price, bl...","[rapid, hiv, antibodi, test, finger, price, bl...","[rapid, hiv, antibodi, test, finger, price, bl...",rapid hiv antibodi test finger price blood day...
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possible hiv symptoms i had unprotected sex wi...,"[possible, hiv, symptoms, i, had, unprotected,...","[possible, hiv, symptoms, unprotected, sex, ma...","[possibl, hiv, symptom, unprotect, sex, man, f...","[possibl, hiv, symptom, unprotect, sex, man, f...",possibl hiv symptom unprotect sex man femal we...
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtears on penis and oral sex i just got a ...,"[microtears, on, penis, and, oral, sex, i, jus...","[microtears, penis, oral, sex, got, blowjob, l...","[microtear, peni, oral, sex, got, blowjob, lik...","[microtear, peni, oral, sex, got, blowjob, lik...",microtear peni oral sex got blowjob like secon...
...,...,...,...,...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get tested share your hiv status pro...,"[carexocom, get, tested, share, your, hiv, sta...","[carexocom, get, tested, share, hiv, status, p...","[carexocom, get, test, share, hiv, statu, prot...","[carexocom, get, test, share, hiv, statu, prot...",carexocom get test share hiv statu protect other
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away from mother for missing docto...,"[child, taken, away, from, mother, for, missin...","[child, taken, away, mother, missing, doctors,...","[child, taken, away, mother, miss, doctor, app...","[child, taken, away, mother, miss, doctor, app...",child taken away mother miss doctor appoint hi...
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test not for those squeamish...,"[safe, sex, hiv, home, test, not, for, those, ...","[safe, sex, hiv, home, test, squeamish, blood]","[safe, sex, hiv, home, test, squeamish, blood]","[safe, sex, hiv, home, test, squeamish, blood]",safe sex hiv home test squeamish blood
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposure to hiv has me breaking out two days a...,"[exposure, to, hiv, has, me, breaking, out, tw...","[exposure, hiv, breaking, two, days, ago, outs...","[exposur, hiv, break, two, day, ago, outsid, s...","[exposur, hiv, break, two, day, ago, outsid, s...",exposur hiv break two day ago outsid store smo...


In [37]:
df_hiv_post.columns

Index(['ID', 'Post_Text', 'Total_Comments', 'Score', 'Post_Created',
       'clean_text', 'tokenization', 'remove_stopwords', 'stemming',
       'lemmatizing', 'post-text'],
      dtype='object')

In [39]:
df_hiv_post_final = df_hiv_post[["ID", "Post_Text", "Total_Comments", "Score", "Post_Created", "post-text"]]
df_hiv_post_final

,ID,Post_Text,Total_Comments,Score,Post_Created,post-text
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,hope help
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk wonder risk bleed hemorrhoid use toil...
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,rapid hiv antibodi test finger price blood day...
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possibl hiv symptom unprotect sex man femal we...
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtear peni oral sex got blowjob like secon...
...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get test share hiv statu protect other
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away mother miss doctor appoint hi...
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test squeamish blood
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposur hiv break two day ago outsid store smo...


In [40]:
#Getting sentiment scores
from textblob import TextBlob

In [41]:
## Define a function which can be applied to calculate the score for the whole dataset
 
def senti(x):
    return TextBlob(x).sentiment  
 
df_hiv_post_final['senti_score'] = df_hiv_post_final['post-text'].apply(senti)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
df_hiv_post_final
#As it can be observed there are two scores: the first score is sentiment polarity which tells if the sentiment is positive or negative 
  #and the second score is subjectivity score to tell how subjective is the text.
#The score indicates how negative or positive the overall text analyzed is. Anything below a score of -0.05 we tag as negative 
  #and anything above 0.05 we tag as positive. Anything in between inclusively, we tag as neutral.

,ID,Post_Text,Total_Comments,Score,Post_Created,post-text,senti_score
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,hope help,"(0.0, 0.0)"
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk wonder risk bleed hemorrhoid use toil...,"(-0.022222222222222223, 0.03333333333333333)"
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,rapid hiv antibodi test finger price blood day...,"(0.5, 0.6666666666666667)"
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possibl hiv symptom unprotect sex man femal we...,"(0.03333333333333333, 0.3)"
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtear peni oral sex got blowjob like secon...,"(0.04523809523809524, 0.3119047619047619)"
...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get test share hiv statu protect other,"(-0.125, 0.375)"
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away mother miss doctor appoint hi...,"(0.13636363636363635, 0.5)"
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test squeamish blood,"(0.5, 0.5)"
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposur hiv break two day ago outsid store smo...,"(0.23095238095238094, 0.4707671957671957)"


In [44]:
df_hiv_post_final["senti_score"][1]

Sentiment(polarity=-0.022222222222222223, subjectivity=0.03333333333333333)

In [45]:
df_hiv_post_final["polarity"] = [t[0] for t in df_hiv_post_final["senti_score"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_hiv_post_final["polarity"][1]

-0.022222222222222223

In [53]:
df_hiv_post_final["type"] = ["negative" if t<-0.05 else "positive" if t>0.05 else "nuetral" if t==0.05 else " " for t in df_hiv_post_final["polarity"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
df_hiv_post_final

,ID,Post_Text,Total_Comments,Score,Post_Created,post-text,senti_score,polarity,type
0,txhj07,I hope this helps,0,4,2022-04-06 04:39:35-04:00,hope help,"(0.0, 0.0)",0.000000,
1,tx850u,HIV Risk I was just wondering what is my risk ...,13,0,2022-04-05 19:24:29-04:00,hiv risk wonder risk bleed hemorrhoid use toil...,"(-0.022222222222222223, 0.03333333333333333)",-0.022222,
2,twx61z,i did a rapid hiv 1/2 antibody test (finger pr...,8,0,2022-04-05 11:10:19-04:00,rapid hiv antibodi test finger price blood day...,"(0.5, 0.6666666666666667)",0.500000,positive
3,twwkp4,Possible HIV symptoms I had unprotected sex wi...,11,0,2022-04-05 10:43:58-04:00,possibl hiv symptom unprotect sex man femal we...,"(0.03333333333333333, 0.3)",0.033333,
4,tx45ul,Microtears on penis and oral sex I just got a ...,1,0,2022-04-05 16:23:25-04:00,microtear peni oral sex got blowjob like secon...,"(0.04523809523809524, 0.3119047619047619)",0.045238,
...,...,...,...,...,...,...,...,...,...
840,1119m0,"CareXO.com: Get tested. Share your HIV status,...",0,0,2012-10-06 03:59:27-04:00,carexocom get test share hiv statu protect other,"(-0.125, 0.375)",-0.125000,negative
841,19e0g9,Child taken away from mother for missing docto...,0,0,2013-02-28 04:40:43-05:00,child taken away mother miss doctor appoint hi...,"(0.13636363636363635, 0.5)",0.136364,positive
842,1dnd7v,Safe Sex (HIV Home Test) - Not for Those Squea...,4,0,2013-05-03 19:04:51-04:00,safe sex hiv home test squeamish blood,"(0.5, 0.5)",0.500000,positive
843,1m4eey,Exposure to hiv has me freaking out Two days a...,11,0,2013-09-10 14:54:22-04:00,exposur hiv break two day ago outsid store smo...,"(0.23095238095238094, 0.4707671957671957)",0.230952,positive


In [49]:
-0.022222 < -0.05 

False

In [50]:
-0.022222 > -0.05 

True

In [55]:
-0.022222 == -0.05 

False

In [56]:
df_hiv_post_final.to_csv('HIV_POST.csv',index=False)